In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## Loading data fashion mnist

In [2]:
(train_img,train_labels),(test_img,test_labels)=keras.datasets.fashion_mnist.load_data()

## Scaling down the images 

In [3]:
train_img=train_img/255.0
test_img= test_img/255.0

## Checking the pixel

In [4]:
train_img.shape

(60000, 28, 28)

the pixels are 28 by 28

## Reshape the images

In [5]:
train_img=train_img.reshape(len(train_img),28,28,1)
test_img=test_img.reshape(len(test_img),28,28,1)

## Building model for CNN

In [6]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

## Keras tuner

In [7]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

## Initialized the Randomsearch 

In [8]:
tuner_search=RandomSearch(build_model,objective="accuracy",max_trials=2, directory="output",project_name="Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Fashion/oracle.json


In [9]:
tuner_search.search(train_img,train_labels,epochs=2)

Trial 2 Complete [00h 07m 10s]
accuracy: 0.9090333580970764

Best accuracy So Far: 0.9090333580970764
Total elapsed time: 00h 25m 24s
INFO:tensorflow:Oracle triggered exit


In [20]:
tuner_search.results_summary()

Results summary
Results in output/Fashion
Showing 10 best trials
Objective(name='accuracy', direction='max')
Trial summary
Hyperparameters:
conv_1_filter: 32
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 96
learning_rate: 0.001
Score: 0.9090333580970764
Trial summary
Hyperparameters:
conv_1_filter: 96
conv_1_kernel: 3
conv_2_filter: 32
conv_2_kernel: 5
dense_1_units: 48
learning_rate: 0.001
Score: 0.9051499962806702


In [23]:
model = keras.Sequential([
    keras.layers.Conv2D(
        filters=32,
        kernel_size=3,
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=32,
        kernel_size=5,
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=96,
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

In [24]:
model.compile(optimizer=keras.optimizers.Adam(.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
model.fit(train_img, train_labels, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 218s 115ms/step - loss: 0.4996 - accuracy: 0.8177
Epoch 2/2
1875/1875 [==============================] - 213s 113ms/step - loss: 0.2455 - accuracy: 0.9085
